# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd

In [2]:
ttt = pd.read_csv('tic-tac-toe.csv')

In [3]:
ttt.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [4]:
#ttt = ttt.astype('category')

In [5]:
ttt_dummies = pd.get_dummies(ttt)
ttt_dummies["class"] = ttt_dummies["class"].astype(int)

In [7]:
ttt_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 28 columns):
class    958 non-null int64
TL_b     958 non-null uint8
TL_o     958 non-null uint8
TL_x     958 non-null uint8
TM_b     958 non-null uint8
TM_o     958 non-null uint8
TM_x     958 non-null uint8
TR_b     958 non-null uint8
TR_o     958 non-null uint8
TR_x     958 non-null uint8
ML_b     958 non-null uint8
ML_o     958 non-null uint8
ML_x     958 non-null uint8
MM_b     958 non-null uint8
MM_o     958 non-null uint8
MM_x     958 non-null uint8
MR_b     958 non-null uint8
MR_o     958 non-null uint8
MR_x     958 non-null uint8
BL_b     958 non-null uint8
BL_o     958 non-null uint8
BL_x     958 non-null uint8
BM_b     958 non-null uint8
BM_o     958 non-null uint8
BM_x     958 non-null uint8
BR_b     958 non-null uint8
BR_o     958 non-null uint8
BR_x     958 non-null uint8
dtypes: int64(1), uint8(27)
memory usage: 32.8 KB


In [8]:
cols = ttt_dummies.columns
cols

Index(['class', 'TL_b', 'TL_o', 'TL_x', 'TM_b', 'TM_o', 'TM_x', 'TR_b', 'TR_o',
       'TR_x', 'ML_b', 'ML_o', 'ML_x', 'MM_b', 'MM_o', 'MM_x', 'MR_b', 'MR_o',
       'MR_x', 'BL_b', 'BL_o', 'BL_x', 'BM_b', 'BM_o', 'BM_x', 'BR_b', 'BR_o',
       'BR_x'],
      dtype='object')

In [9]:
'''
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(ttt[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']]).toarray())
# merge with main df on key values
ttt_OHE = ttt.join(enc_df)
ttt_OHE.drop(columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], inplace=True)'''

In [9]:
y_df = ttt_dummies["class"]

In [10]:
X_df = ttt_dummies.drop(columns = 'class')

In [11]:
X_df.head()

,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,ML_b,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
4,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0


In [12]:
y_df.head()

0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

In [14]:
'''from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()
std_scaler
# fit and transform the data
X_std = pd.DataFrame(std_scaler.fit_transform(X_df))
y_std = pd.DataFrame(std_scaler.fit_transform(y_df))
X_std.head()


Error and no need to reshape since dummies are 0 - 1'''


'from sklearn.preprocessing import StandardScaler\n\n# create a scaler object\nstd_scaler = StandardScaler()\nstd_scaler\n# fit and transform the data\nX_std = pd.DataFrame(std_scaler.fit_transform(X_df))\ny_std = pd.DataFrame(std_scaler.fit_transform(y_df))\nX_std.head()\n\n\nError and no need to reshape since dummies are 0 - 1'

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [15]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [16]:
x_train, y_train, x_test, y_test = train_test_split(X_df, y_df, random_state=42)

In [17]:
import numpy as np
x_train, x_test, y_train, y_test = [np.array(elemnt, dtype = float) for elemnt in [x_train, y_train, x_test, y_test]]

print(f"{x_train.shape}{type(x_train)} \t {y_train.shape}{type(y_train)}")
print(f"{x_test.shape}{type(x_test)} \t {y_test.shape}{type(y_test)}")

(718, 27)<class 'numpy.ndarray'> 	 (718,)<class 'numpy.ndarray'>
(240, 27)<class 'numpy.ndarray'> 	 (240,)<class 'numpy.ndarray'>


In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [19]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
#              validation_data=(x_test, y_test)

In [20]:
history = model.fit(x_train, 
                    y_train, 
                    batch_size=64, 
                    epochs=5, 
                    validation_data=(x_test, y_test))

Train on 718 samples, validate on 240 samples
Epoch 1/5
718/718 [==============================] - 0s 402us/sample - loss: 0.6583 - accuracy: 0.6379 - val_loss: 0.6128 - val_accuracy: 0.6917
Epoch 2/5
718/718 [==============================] - 0s 26us/sample - loss: 0.6142 - accuracy: 0.6490 - val_loss: 0.5896 - val_accuracy: 0.7167
Epoch 3/5
718/718 [==============================] - 0s 25us/sample - loss: 0.5865 - accuracy: 0.7159 - val_loss: 0.5749 - val_accuracy: 0.7625
Epoch 4/5
718/718 [==============================] - 0s 26us/sample - loss: 0.5638 - accuracy: 0.7493 - val_loss: 0.5617 - val_accuracy: 0.7583
Epoch 5/5
718/718 [==============================] - 0s 26us/sample - loss: 0.5481 - accuracy: 0.7730 - val_loss: 0.5508 - val_accuracy: 0.7708


In [21]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

240/240 [==============================] - 0s 17us/sample - loss: 0.5508 - accuracy: 0.7708
0.5507821639378866 0.7708333


In [22]:
model.save('tic-tac-toe.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [35]:
model = tf.keras.models.load_model('tic-tac-toe.model')

predictor = model.predict(x_test)

In [36]:
'''This are probability distritutions. A tensor that need to be unrooled'''
print(predictor)

[[0.04999027 0.95000976]
 [0.20968665 0.79031336]
 [0.33477554 0.66522443]
 [0.2074044  0.79259557]
 [0.79489243 0.20510754]
 [0.04026308 0.9597369 ]
 [0.03650827 0.96349174]
 [0.8629302  0.13706985]
 [0.51573306 0.4842669 ]
 [0.19377154 0.80622846]
 [0.08924932 0.91075075]
 [0.26417413 0.7358259 ]
 [0.01300398 0.98699594]
 [0.7823133  0.2176867 ]
 [0.29339403 0.7066059 ]
 [0.02388973 0.9761103 ]
 [0.4880981  0.5119019 ]
 [0.27764538 0.7223546 ]
 [0.9238422  0.07615778]
 [0.7328588  0.26714122]
 [0.4322675  0.5677325 ]
 [0.06277683 0.93722314]
 [0.5017907  0.49820924]
 [0.15234002 0.84766   ]
 [0.6419815  0.35801855]
 [0.594491   0.40550902]
 [0.04268461 0.9573154 ]
 [0.09386809 0.9061319 ]
 [0.31795594 0.6820441 ]
 [0.20709097 0.792909  ]
 [0.00649907 0.9935009 ]
 [0.08735601 0.91264397]
 [0.12307016 0.8769298 ]
 [0.12337381 0.8766262 ]
 [0.20838992 0.79161006]
 [0.14160213 0.85839784]
 [0.5482923  0.45170766]
 [0.3325221  0.66747785]
 [0.06465618 0.93534386]
 [0.6077055  0.3922945 ]


In [97]:
'''
1 means that player X will win,
0 means that player Y will win or result will be a draw.
'''

#Prediction 25 results
predictions = [np.argmax(predictor[i]) for i in range(y_test.shape[0])]
print(predictions)

predictions_df = pd.DataFrame(predictions)
predictions_df['test'] = y_test.astype(int)

correct = 0
fail = 0

for val in range(24):   
    if predictions_df.loc[predictions_df[0] == predictions_df['test']]:
        correct += 1
    else:
        fail += 1
        
print(f'In 25 attemps, the model was spon on {correct} times and failed {fail} times')

[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## Step 4: Improve Your Model
Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
Adjust the learning rate when you compile the model. This means you will create a custom tf.keras.optimizers.Adam instance where you specify the learning rate you want. Then pass the instance to model.compile as the optimizer.
tf.keras.optimizers.Adam reference.
Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [162]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [163]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0008), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
#              validation_data=(x_test, y_test)

In [164]:
history = model.fit(x_train, 
                    y_train, 
                    batch_size=50, 
                    epochs=20, 
                    validation_data=(x_test, y_test))

Train on 718 samples, validate on 240 samples
Epoch 1/20
718/718 [==============================] - 0s 448us/sample - loss: 0.6603 - accuracy: 0.6212 - val_loss: 0.6092 - val_accuracy: 0.6875
Epoch 2/20
718/718 [==============================] - 0s 40us/sample - loss: 0.6162 - accuracy: 0.6797 - val_loss: 0.5732 - val_accuracy: 0.7292
Epoch 3/20
718/718 [==============================] - 0s 40us/sample - loss: 0.5665 - accuracy: 0.7368 - val_loss: 0.5377 - val_accuracy: 0.7750
Epoch 4/20
718/718 [==============================] - 0s 43us/sample - loss: 0.5262 - accuracy: 0.7869 - val_loss: 0.5093 - val_accuracy: 0.7750
Epoch 5/20
718/718 [==============================] - 0s 38us/sample - loss: 0.4808 - accuracy: 0.8398 - val_loss: 0.4700 - val_accuracy: 0.8292
Epoch 6/20
718/718 [==============================] - 0s 38us/sample - loss: 0.4327 - accuracy: 0.8955 - val_loss: 0.4190 - val_accuracy: 0.9083
Epoch 7/20
718/718 [==============================] - 0s 40us/sample - loss: 0.3851

**Which approach(es) did you find helpful to improve your model performance?**

I found that there is a max number of epochs needed to get the perfect accuracy. But I can't get the loss smaller that 0.3, no matter what param I change